In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/ucb_final_proj/images_all2.zip

--2020-11-22 19:52:24--  https://storage.googleapis.com/ucb_final_proj/images_all2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4945526734 (4.6G) [application/zip]
Saving to: ‘images_all2.zip’

images_all2.zip     100%[===================>]   4.61G  75.8MB/s    in 95s     

2020-11-22 19:53:59 (49.8 MB/s) - ‘images_all2.zip’ saved [4945526734/4945526734]



In [ ]:
import os
import zipfile

zip_path = 'images_all2.zip'
zip_ref = zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
!pip install split_folders

In [ ]:
import splitfolders

input_folder = "images_all"
output = "splited_images"
splitfolders.ratio(input_folder, output, seed=42, ratio=(.8, .1, .1))

Copying files: 98547 files [02:21, 698.74 files/s]


In [ ]:
train_path = 'splited_images/train'
valid_path = 'splited_images/val'
test_path = 'splited_images/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
import random

datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

In [ ]:
train_batches=datagen.flow_from_directory(directory=train_path,target_size=(224,224),batch_size=64)
valid_batches=datagen.flow_from_directory(directory=valid_path,target_size=(224,224),batch_size=64)
test_bathes=datagen.flow_from_directory(directory=test_path,target_size=(224,224),batch_size=20,shuffle=False)

Found 78798 images belonging to 101 classes.
Found 9809 images belonging to 101 classes.
Found 9940 images belonging to 101 classes.


## **Modify Model**

In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

#fine tuning the model and take all layers up until the 6th to the last
#aks remove the last 5 layers
x = mobile.layers[-6].output

#'softmax will give us a probability distribution of each class
# x is the previous layers we store from 
output = Dense(units=101,activation='softmax')(x)

In [ ]:
model = tf.keras.Model(inputs=mobile.input, outputs=output)

In [ ]:
#make last 23 layers not trainable. there were 88 layers in the original mobile model
for layer in model.layers[:-23]:
  layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [ ]:
model.fit(x=train_batches,validation_data=valid_batches,epochs=30)

Epoch 1/30
1232/1232 [==============================] - 460s 373ms/step - loss: 1.2483 - accuracy: 0.6862 - val_loss: 1.4012 - val_accuracy: 0.6432
Epoch 2/30
1232/1232 [==============================] - 448s 363ms/step - loss: 0.9780 - accuracy: 0.7529 - val_loss: 1.2848 - val_accuracy: 0.6658
Epoch 3/30
1232/1232 [==============================] - 446s 362ms/step - loss: 0.7748 - accuracy: 0.8070 - val_loss: 1.2608 - val_accuracy: 0.6680
Epoch 4/30
1232/1232 [==============================] - 445s 361ms/step - loss: 0.6132 - accuracy: 0.8534 - val_loss: 1.2598 - val_accuracy: 0.6757
Epoch 5/30
1232/1232 [==============================] - 444s 360ms/step - loss: 0.4798 - accuracy: 0.8922 - val_loss: 1.2655 - val_accuracy: 0.6753
Epoch 6/30
1232/1232 [==============================] - 441s 358ms/step - loss: 0.3659 - accuracy: 0.9284 - val_loss: 1.2722 - val_accuracy: 0.6778
Epoch 7/30
1232/1232 [==============================] - 440s 357ms/step - loss: 0.2726 - accuracy: 0.9542 - val_

In [ ]:
model.save('MobileNet.h5')

In [ ]:
model_loss, model_accuracy = model.evaluate(x=test_bathes, verbose=2)
print(
    f"MobileNet Model - Loss: {model_loss}, Accuracy: {model_accuracy}")

497/497 - 58s - loss: 1.9139 - accuracy: 0.6713
MobileNet Model - Loss: 1.913917064666748, Accuracy: 0.6713279485702515
